In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from fvcore.nn import FlopCountAnalysis
from  fvcore.nn import flop_count_table

In [2]:
from fvcore.nn import parameter_count

# VGG Related

In [4]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)


In [5]:
net_glob = VGG('VGG11').to("cuda")
inp = torch.randn([1,3,32,32]).cuda()

# log_probs = net_glob(inp)
# inp = torch.randn([3,32,32]).cuda()
flops_cnn = FlopCountAnalysis(net_glob, inp)
print("Total FLOPs: " + str(flops_cnn.total()))
print(flop_count_table(flops_cnn))

Unsupported operator aten::max_pool2d encountered 5 time(s)
Unsupported operator aten::avg_pool2d encountered 1 time(s)


Total FLOPs: 153527296
| module                | #parameters or shape   | #flops    |
|:----------------------|:-----------------------|:----------|
| model                 | 9.231M                 | 0.154G    |
|  features             |  9.226M                |  0.154G   |
|   features.0          |   1.792K               |   1.769M  |
|    features.0.weight  |    (64, 3, 3, 3)       |           |
|    features.0.bias    |    (64,)               |           |
|   features.1          |   0.128K               |   0.328M  |
|    features.1.weight  |    (64,)               |           |
|    features.1.bias    |    (64,)               |           |
|   features.4          |   73.856K              |   18.874M |
|    features.4.weight  |    (128, 64, 3, 3)     |           |
|    features.4.bias    |    (128,)              |           |
|   features.5          |   0.256K               |   0.164M  |
|    features.5.weight  |    (128,)              |           |
|    features.5.bias    |    (12

In [ ]:
"""
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (17): ReLU(inplace=True)
    (18): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (19): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (20): ReLU(inplace=True)
    (21): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (22): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (23): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (24): ReLU(inplace=True)
    (25): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (26): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (27): ReLU(inplace=True)
    (28): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (29): AvgPool2d(kernel_size=1, stride=1, padding=0)
  )
  (classifier): Linear(in_features=512, out_features=10, bias=True)
)
"""

# CNN CIFAR

In [3]:
class Full_CNNCifar(nn.Module):
    def __init__(self, num_classes):
        super(Full_CNNCifar, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 100)
        self.fc3 = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net_glob = Full_CNNCifar(num_classes = 10).to("cuda")
inp = torch.randn([1,3,32,32]).cuda()

# log_probs = net_glob(inp)
# inp = torch.randn([3,32,32]).cuda()
flops_cnn = FlopCountAnalysis(net_glob, inp)
print("Total FLOPs: " + str(flops_cnn.total()))
print(flop_count_table(flops_cnn))

Unsupported operator aten::max_pool2d encountered 2 time(s)


Total FLOPs: 653800
| module         | #parameters or shape   | #flops   |
|:---------------|:-----------------------|:---------|
| model          | 64.102K                | 0.654M   |
|  conv1         |  0.456K                |  0.353M  |
|   conv1.weight |   (6, 3, 5, 5)         |          |
|   conv1.bias   |   (6,)                 |          |
|  conv2         |  2.416K                |  0.24M   |
|   conv2.weight |   (16, 6, 5, 5)        |          |
|   conv2.bias   |   (16,)                |          |
|  fc1           |  48.12K                |  48K     |
|   fc1.weight   |   (120, 400)           |          |
|   fc1.bias     |   (120,)               |          |
|  fc2           |  12.1K                 |  12K     |
|   fc2.weight   |   (100, 120)           |          |
|   fc2.bias     |   (100,)               |          |
|  fc3           |  1.01K                 |  1K      |
|   fc3.weight   |   (10, 100)            |          |
|   fc3.bias     |   (10,)                |  

In [4]:
FlopCountAnalysis(net_glob, inp).total()

Unsupported operator aten::max_pool2d encountered 2 time(s)


653800

In [5]:
parameter_count(net_glob)

defaultdict(int,
            {'': 64102,
             'conv1': 456,
             'conv1.weight': 450,
             'conv1.bias': 6,
             'conv2': 2416,
             'conv2.weight': 2400,
             'conv2.bias': 16,
             'fc1': 48120,
             'fc1.weight': 48000,
             'fc1.bias': 120,
             'fc2': 12100,
             'fc2.weight': 12000,
             'fc2.bias': 100,
             'fc3': 1010,
             'fc3.weight': 1000,
             'fc3.bias': 10})

In [6]:
FlopCountAnalysis(net_glob, inp).total()

Unsupported operator aten::max_pool2d encountered 2 time(s)


653800

In [6]:
# Mode 1
# 6->5, 16 -> 12, 120 ->100, 100-> 84 
class CNNCifar_1(nn.Module):
    def __init__(self, num_classes):
        super(CNNCifar_1, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 12, 5)
        self.fc1 = nn.Linear(12 * 5 * 5, 100)
        self.fc2 = nn.Linear(100, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 12 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net_glob = CNNCifar_1(num_classes = 10).to("cuda")
inp = torch.randn([1,3,32,32]).cuda()

# log_probs = net_glob(inp)
# inp = torch.randn([3,32,32]).cuda()
flops_cnn = FlopCountAnalysis(net_glob, inp)
print("Total FLOPs: " + str(flops_cnn.total()))
print(flop_count_table(flops_cnn))

Unsupported operator aten::max_pool2d encountered 2 time(s)


Total FLOPs: 483240
| module         | #parameters or shape   | #flops   |
|:---------------|:-----------------------|:---------|
| model          | 41.326K                | 0.483M   |
|  conv1         |  0.38K                 |  0.294M  |
|   conv1.weight |   (5, 3, 5, 5)         |          |
|   conv1.bias   |   (5,)                 |          |
|  conv2         |  1.512K                |  0.15M   |
|   conv2.weight |   (12, 5, 5, 5)        |          |
|   conv2.bias   |   (12,)                |          |
|  fc1           |  30.1K                 |  30K     |
|   fc1.weight   |   (100, 300)           |          |
|   fc1.bias     |   (100,)               |          |
|  fc2           |  8.484K                |  8.4K    |
|   fc2.weight   |   (84, 100)            |          |
|   fc2.bias     |   (84,)                |          |
|  fc3           |  0.85K                 |  0.84K   |
|   fc3.weight   |   (10, 84)             |          |
|   fc3.bias     |   (10,)                |  

In [7]:
parameter_count(net_glob)

defaultdict(int,
            {'': 41326,
             'conv1': 380,
             'conv1.weight': 375,
             'conv1.bias': 5,
             'conv2': 1512,
             'conv2.weight': 1500,
             'conv2.bias': 12,
             'fc1': 30100,
             'fc1.weight': 30000,
             'fc1.bias': 100,
             'fc2': 8484,
             'fc2.weight': 8400,
             'fc2.bias': 84,
             'fc3': 850,
             'fc3.weight': 840,
             'fc3.bias': 10})

In [8]:
FlopCountAnalysis(net_glob, inp).total()

Unsupported operator aten::max_pool2d encountered 2 time(s)


483240

In [11]:
# Mode 2
# 6->5, 16 -> 12, _________, 100-> 84 
class CNNCifar1(nn.Module):
    def __init__(self, num_classes):
        super(CNNCifar1, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 12, 5)
        self.fc1 = nn.Linear(12 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 12 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net_glob = CNNCifar1(num_classes = 10).to("cuda")
inp = torch.randn([1,3,32,32]).cuda()

# log_probs = net_glob(inp)
# inp = torch.randn([3,32,32]).cuda()
flops_cnn = FlopCountAnalysis(net_glob, inp)
print("Total FLOPs: " + str(flops_cnn.total()))
print(flop_count_table(flops_cnn))

Unsupported operator aten::max_pool2d encountered 2 time(s)


Total FLOPs: 490920
| module         | #parameters or shape   | #flops   |
|:---------------|:-----------------------|:---------|
| model          | 49.026K                | 0.491M   |
|  conv1         |  0.38K                 |  0.294M  |
|   conv1.weight |   (5, 3, 5, 5)         |          |
|   conv1.bias   |   (5,)                 |          |
|  conv2         |  1.512K                |  0.15M   |
|   conv2.weight |   (12, 5, 5, 5)        |          |
|   conv2.bias   |   (12,)                |          |
|  fc1           |  36.12K                |  36K     |
|   fc1.weight   |   (120, 300)           |          |
|   fc1.bias     |   (120,)               |          |
|  fc2           |  10.164K               |  10.08K  |
|   fc2.weight   |   (84, 120)            |          |
|   fc2.bias     |   (84,)                |          |
|  fc3           |  0.85K                 |  0.84K   |
|   fc3.weight   |   (10, 84)             |          |
|   fc3.bias     |   (10,)                |  

In [12]:
# Mode 3
# 6->5, 16 -> 12, ___________________________
class CNNCifar_3(nn.Module):
    def __init__(self, num_classes):
        super(CNNCifar_3, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 12, 5)
        self.fc1 = nn.Linear(12 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 100)
        self.fc3 = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 12 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net_glob = CNNCifar_3(num_classes = 10).to("cuda")
inp = torch.randn([1,3,32,32]).cuda()

# log_probs = net_glob(inp)
# inp = torch.randn([3,32,32]).cuda()
flops_cnn = FlopCountAnalysis(net_glob, inp)
print("Total FLOPs: " + str(flops_cnn.total()))
print(flop_count_table(flops_cnn))

Unsupported operator aten::max_pool2d encountered 2 time(s)


Total FLOPs: 493000
| module         | #parameters or shape   | #flops   |
|:---------------|:-----------------------|:---------|
| model          | 51.122K                | 0.493M   |
|  conv1         |  0.38K                 |  0.294M  |
|   conv1.weight |   (5, 3, 5, 5)         |          |
|   conv1.bias   |   (5,)                 |          |
|  conv2         |  1.512K                |  0.15M   |
|   conv2.weight |   (12, 5, 5, 5)        |          |
|   conv2.bias   |   (12,)                |          |
|  fc1           |  36.12K                |  36K     |
|   fc1.weight   |   (120, 300)           |          |
|   fc1.bias     |   (120,)               |          |
|  fc2           |  12.1K                 |  12K     |
|   fc2.weight   |   (100, 120)           |          |
|   fc2.bias     |   (100,)               |          |
|  fc3           |  1.01K                 |  1K      |
|   fc3.weight   |   (10, 100)            |          |
|   fc3.bias     |   (10,)                |  

In [ ]:
FlopCountAnalysis(net_glob, inp).total()
parameter_count(net_glob)

In [ ]:
# Mode 4
#  _______,________, 120 ->100, 100-> 84 
class CNNCifar4(nn.Module):
    def __init__(self, num_classes):
        super(CNNCifar4, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 12, 5)
        self.fc1 = nn.Linear(12 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 12 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net_glob = CNNCifar4(num_classes = 10).to("cuda")
inp = torch.randn([1,3,32,32]).cuda()

# log_probs = net_glob(inp)
# inp = torch.randn([3,32,32]).cuda()
flops_cnn = FlopCountAnalysis(net_glob, inp)
print("Total FLOPs: " + str(flops_cnn.total()))
print(flop_count_table(flops_cnn))

In [ ]:
FlopCountAnalysis(net_glob, inp).total()
parameter_count(net_glob)

In [31]:
# Mode 1
class CNNCifar2(nn.Module):
    def __init__(self, num_classes):
        super(CNNCifar2, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 12, 5)
        self.fc1 = nn.Linear(12 * 5 * 5, 100)
        self.fc2 = nn.Linear(100, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 12 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [32]:
net_glob = CNNCifar2(num_classes = 10).to("cuda")
inp = torch.randn([1,3,32,32]).cuda()

# log_probs = net_glob(inp)
# inp = torch.randn([3,32,32]).cuda()
flops_cnn = FlopCountAnalysis(net_glob, inp)
print("Total FLOPs: " + str(flops_cnn.total()))
print(flop_count_table(flops_cnn))

Unsupported operator aten::max_pool2d encountered 2 time(s)


Total FLOPs: 483240
| module         | #parameters or shape   | #flops   |
|:---------------|:-----------------------|:---------|
| model          | 41.326K                | 0.483M   |
|  conv1         |  0.38K                 |  0.294M  |
|   conv1.weight |   (5, 3, 5, 5)         |          |
|   conv1.bias   |   (5,)                 |          |
|  conv2         |  1.512K                |  0.15M   |
|   conv2.weight |   (12, 5, 5, 5)        |          |
|   conv2.bias   |   (12,)                |          |
|  fc1           |  30.1K                 |  30K     |
|   fc1.weight   |   (100, 300)           |          |
|   fc1.bias     |   (100,)               |          |
|  fc2           |  8.484K                |  8.4K    |
|   fc2.weight   |   (84, 100)            |          |
|   fc2.bias     |   (84,)                |          |
|  fc3           |  0.85K                 |  0.84K   |
|   fc3.weight   |   (10, 84)             |          |
|   fc3.bias     |   (10,)                |  

In [21]:
# Mode 2
class CNNCifar3(nn.Module):
    def __init__(self, num_classes):
        super(CNNCifar3, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 12, 5)
        self.fc1 = nn.Linear(12 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 12 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [22]:
net_glob = CNNCifar3(num_classes = 10).to("cuda")
inp = torch.randn([1,3,32,32]).cuda()

# log_probs = net_glob(inp)
# inp = torch.randn([3,32,32]).cuda()
flops_cnn = FlopCountAnalysis(net_glob, inp)
print("Total FLOPs: " + str(flops_cnn.total()))
print(flop_count_table(flops_cnn))

Unsupported operator aten::max_pool2d encountered 2 time(s)


Total FLOPs: 490920
| module         | #parameters or shape   | #flops   |
|:---------------|:-----------------------|:---------|
| model          | 49.026K                | 0.491M   |
|  conv1         |  0.38K                 |  0.294M  |
|   conv1.weight |   (5, 3, 5, 5)         |          |
|   conv1.bias   |   (5,)                 |          |
|  conv2         |  1.512K                |  0.15M   |
|   conv2.weight |   (12, 5, 5, 5)        |          |
|   conv2.bias   |   (12,)                |          |
|  fc1           |  36.12K                |  36K     |
|   fc1.weight   |   (120, 300)           |          |
|   fc1.bias     |   (120,)               |          |
|  fc2           |  10.164K               |  10.08K  |
|   fc2.weight   |   (84, 120)            |          |
|   fc2.bias     |   (84,)                |          |
|  fc3           |  0.85K                 |  0.84K   |
|   fc3.weight   |   (10, 84)             |          |
|   fc3.bias     |   (10,)                |  

In [23]:
# Mode 3
class CNNCifar4(nn.Module):
    def __init__(self, num_classes):
        super(CNNCifar4, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [24]:
net_glob = CNNCifar4(num_classes = 10).to("cuda")
inp = torch.randn([1,3,32,32]).cuda()

# log_probs = net_glob(inp)
# inp = torch.randn([3,32,32]).cuda()
flops_cnn = FlopCountAnalysis(net_glob, inp)
print("Total FLOPs: " + str(flops_cnn.total()))
print(flop_count_table(flops_cnn))

Unsupported operator aten::max_pool2d encountered 2 time(s)


Total FLOPs: 552920
| module         | #parameters or shape   | #flops   |
|:---------------|:-----------------------|:---------|
| model          | 61.53K                 | 0.553M   |
|  conv1         |  0.38K                 |  0.294M  |
|   conv1.weight |   (5, 3, 5, 5)         |          |
|   conv1.bias   |   (5,)                 |          |
|  conv2         |  2.016K                |  0.2M    |
|   conv2.weight |   (16, 5, 5, 5)        |          |
|   conv2.bias   |   (16,)                |          |
|  fc1           |  48.12K                |  48K     |
|   fc1.weight   |   (120, 400)           |          |
|   fc1.bias     |   (120,)               |          |
|  fc2           |  10.164K               |  10.08K  |
|   fc2.weight   |   (84, 120)            |          |
|   fc2.bias     |   (84,)                |          |
|  fc3           |  0.85K                 |  0.84K   |
|   fc3.weight   |   (10, 84)             |          |
|   fc3.bias     |   (10,)                |  

In [25]:
# Mode 4
class CNNCifar5(nn.Module):
    def __init__(self, num_classes):
        super(CNNCifar5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 100)
        self.fc2 = nn.Linear(100, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [26]:
net_glob = CNNCifar5(num_classes = 10).to("cuda")
inp = torch.randn([1,3,32,32]).cuda()

# log_probs = net_glob(inp)
# inp = torch.randn([3,32,32]).cuda()
flops_cnn = FlopCountAnalysis(net_glob, inp)
print("Total FLOPs: " + str(flops_cnn.total()))
print(flop_count_table(flops_cnn))

Unsupported operator aten::max_pool2d encountered 2 time(s)


Total FLOPs: 642040
| module         | #parameters or shape   | #flops   |
|:---------------|:-----------------------|:---------|
| model          | 52.306K                | 0.642M   |
|  conv1         |  0.456K                |  0.353M  |
|   conv1.weight |   (6, 3, 5, 5)         |          |
|   conv1.bias   |   (6,)                 |          |
|  conv2         |  2.416K                |  0.24M   |
|   conv2.weight |   (16, 6, 5, 5)        |          |
|   conv2.bias   |   (16,)                |          |
|  fc1           |  40.1K                 |  40K     |
|   fc1.weight   |   (100, 400)           |          |
|   fc1.bias     |   (100,)               |          |
|  fc2           |  8.484K                |  8.4K    |
|   fc2.weight   |   (84, 100)            |          |
|   fc2.bias     |   (84,)                |          |
|  fc3           |  0.85K                 |  0.84K   |
|   fc3.weight   |   (10, 84)             |          |
|   fc3.bias     |   (10,)                |  

In [9]:
class MLP_from_CNN(nn.Module):
    def __init__(self):
        super(MLP_from_CNN, self).__init__()
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x    

In [54]:
net_glob = MLP_from_CNN().to("cuda")
inp = torch.randn(400,1).cuda()
flops1 = FlopCountAnalysis(net_glob, inp)
print(flops1.total())
print(flop_count_table(flops1))

58920
| module       | #parameters or shape   | #flops   |
|:-------------|:-----------------------|:---------|
| model        | 59.134K                | 58.92K   |
|  fc1         |  48.12K                |  48K     |
|   fc1.weight |   (120, 400)           |          |
|   fc1.bias   |   (120,)               |          |
|  fc2         |  10.164K               |  10.08K  |
|   fc2.weight |   (84, 120)            |          |
|   fc2.bias   |   (84,)                |          |
|  fc3         |  0.85K                 |  0.84K   |
|   fc3.weight |   (10, 84)             |          |
|   fc3.bias   |   (10,)                |          |


In [45]:
class CNNCifar2(nn.Module):
    def __init__(self, num_classes):
        super(CNNCifar2, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net_glob = CNNCifar2(num_classes = 10).to("cuda")
inp = torch.randn([1,3,32,32]).cuda()

# log_probs = net_glob(inp)
# inp = torch.randn([3,32,32]).cuda()
flops1 = FlopCountAnalysis(net_glob, inp)
print("Total FLOPs: " + str(flops1.total()))
print(flop_count_table(flops1))

Unsupported operator aten::max_pool2d encountered 2 time(s)


Total FLOPs: 552920
| module         | #parameters or shape   | #flops   |
|:---------------|:-----------------------|:---------|
| model          | 61.53K                 | 0.553M   |
|  conv1         |  0.38K                 |  0.294M  |
|   conv1.weight |   (5, 3, 5, 5)         |          |
|   conv1.bias   |   (5,)                 |          |
|  conv2         |  2.016K                |  0.2M    |
|   conv2.weight |   (16, 5, 5, 5)        |          |
|   conv2.bias   |   (16,)                |          |
|  fc1           |  48.12K                |  48K     |
|   fc1.weight   |   (120, 400)           |          |
|   fc1.bias     |   (120,)               |          |
|  fc2           |  10.164K               |  10.08K  |
|   fc2.weight   |   (84, 120)            |          |
|   fc2.bias     |   (84,)                |          |
|  fc3           |  0.85K                 |  0.84K   |
|   fc3.weight   |   (10, 84)             |          |
|   fc3.bias     |   (10,)                |  

# MLP

In [ ]:
#39200 *4 -> 156800

In [26]:
class MLP(nn.Module):
    def __init__(self, dim_in, dim_hidden, dim_out):
        super(MLP, self).__init__()
        self.layer_input = nn.Linear(dim_in, dim_hidden)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout()
        self.layer_hidden = nn.Linear(dim_hidden, dim_out)

    def forward(self, x):
        x = x.view(-1, x.shape[1]*x.shape[-2]*x.shape[-1])
        x = self.layer_input(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.layer_hidden(x)
        return x


In [7]:
net_glob = MLP2(dim_in=784, dim_hidden= 150, dim_out=10).to("cuda")
inp = torch.randn(784,1).cuda()
inp2 = torch.randn(200,1).cuda()
flops1 = FlopCountAnalysis(net_glob, (inp,inp2))
flops1.total()

## Full network

In [42]:
net_glob = MLP(dim_in=784, dim_hidden= 200, dim_out=10).to("cuda")
inp = torch.randn(784,1).cuda()
flops1 = FlopCountAnalysis(net_glob, inp)
flops1.total()

Unsupported operator aten::mul encountered 2 time(s)


158800

In [43]:
FlopCountAnalysis(net_glob, inp).total()
parameter_count(net_glob)

Unsupported operator aten::mul encountered 2 time(s)


158800

In [44]:
parameter_count(net_glob)

defaultdict(int,
            {'': 159010,
             'layer_input': 157000,
             'layer_input.weight': 156800,
             'layer_input.bias': 200,
             'layer_hidden': 2010,
             'layer_hidden.weight': 2000,
             'layer_hidden.bias': 10})

In [5]:
#GlobalNet 200 neurons in hidden layer
print(flop_count_table(flops1))

| module                | #parameters or shape   | #flops   |
|:----------------------|:-----------------------|:---------|
| model                 | 0.159M                 | 0.159M   |
|  layer_input          |  0.157M                |  0.157M  |
|   layer_input.weight  |   (200, 784)           |          |
|   layer_input.bias    |   (200,)               |          |
|  layer_hidden         |  2.01K                 |  2K      |
|   layer_hidden.weight |   (10, 200)            |          |
|   layer_hidden.bias   |   (10,)                |          |


## Weights Only

In [27]:
class MLP2(nn.Module):
    def __init__(self, dim_in=784, dim_hidden=150, dim_out=10):
        super(MLP2, self).__init__()
        self.layer_input = nn.Linear(dim_in, dim_hidden)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout()
        self.layer_hidden = nn.Linear(dim_hidden, dim_out)

    def forward(self, x):
        x = x.view(-1, x.shape[1]*x.shape[-2]*x.shape[-1])
        x = self.layer_input(x)
        x = self.layer_hidden(x)
        return x


In [28]:
net_glob = MLP2(dim_in=784, dim_hidden= 150, dim_out=10).to("cuda")
inp = torch.randn(784,1).cuda()
flops1 = FlopCountAnalysis(net_glob, inp)
print(flop_count_table(flops1))

| module                | #parameters or shape   | #flops   |
|:----------------------|:-----------------------|:---------|
| model                 | 0.119M                 | 0.119M   |
|  layer_input          |  0.118M                |  0.118M  |
|   layer_input.weight  |   (150, 784)           |          |
|   layer_input.bias    |   (150,)               |          |
|  layer_hidden         |  1.51K                 |  1.5K    |
|   layer_hidden.weight |   (10, 150)            |          |
|   layer_hidden.bias   |   (10,)                |          |


In [21]:
net_glob = MLP3(dim_in=784, dim_hidden= 150, dim_out=10).to("cuda")
inp = torch.randn(784,1).cuda()
flops1 = FlopCountAnalysis(net_glob, inp)
flops1.total()

Unsupported operator aten::mul encountered 2 time(s)
Unsupported operator aten::add_ encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
dropout, layer_hidden, relu


117600

## Fixed SubNetwork, Neuron Pruning

In [48]:
n1 = MLP(dim_in=784, dim_hidden=150, dim_out=10).to("cuda")
inp = torch.randn(784,1).cuda()
flops2 = FlopCountAnalysis(n1, inp).total()

Unsupported operator aten::mul encountered 2 time(s)


In [49]:
parameter_count(n1)

defaultdict(int,
            {'': 119260,
             'layer_input': 117750,
             'layer_input.weight': 117600,
             'layer_input.bias': 150,
             'layer_hidden': 1510,
             'layer_hidden.weight': 1500,
             'layer_hidden.bias': 10})

In [39]:
from fvcore.nn import parameter_count

In [41]:
FlopCountAnalysis(n1, inp).total()

Unsupported operator aten::mul encountered 2 time(s)


119100

In [40]:
parameter_count(n1)

defaultdict(int,
            {'': 119260,
             'layer_input': 117750,
             'layer_input.weight': 117600,
             'layer_input.bias': 150,
             'layer_hidden': 1510,
             'layer_hidden.weight': 1500,
             'layer_hidden.bias': 10})

In [7]:
#N2 N3 N4 150 neurons in hidden layer
print(flop_count_table(flops2))

| module                | #parameters or shape   | #flops   |
|:----------------------|:-----------------------|:---------|
| model                 | 0.119M                 | 0.119M   |
|  layer_input          |  0.118M                |  0.118M  |
|   layer_input.weight  |   (150, 784)           |          |
|   layer_input.bias    |   (150,)               |          |
|  layer_hidden         |  1.51K                 |  1.5K    |
|   layer_hidden.weight |   (10, 150)            |          |
|   layer_hidden.bias   |   (10,)                |          |


In [8]:
n1 = MLP(dim_in=784, dim_hidden=100, dim_out=10).to("cuda")
inp = torch.randn(784,1).cuda()
flops3 = FlopCountAnalysis(n1, inp)
flops3.total()

Unsupported operator aten::mul encountered 2 time(s)


79400

In [9]:
print(flop_count_table(flops3))

| module                | #parameters or shape   | #flops   |
|:----------------------|:-----------------------|:---------|
| model                 | 79.51K                 | 79.4K    |
|  layer_input          |  78.5K                 |  78.4K   |
|   layer_input.weight  |   (100, 784)           |          |
|   layer_input.bias    |   (100,)               |          |
|  layer_hidden         |  1.01K                 |  1K      |
|   layer_hidden.weight |   (10, 100)            |          |
|   layer_hidden.bias   |   (10,)                |          |


| module                | #parameters or shape   | #flops   |
|:----------------------|:-----------------------|:---------|
| model                 | 79.51K                 | 79.4K    |
|  layer_input          |  78.5K                 |  78.4K   |
|   layer_input.weight  |   (100, 784)           |          |
|   layer_input.bias    |   (100,)               |          |
|  layer_hidden         |  1.01K                 |  1K      |
|   layer_hidden.weight |   (10, 100)            |          |
|   layer_hidden.bias   |   (10,)                |          |


In [5]:
setting_array = [8,0,0,2]

In [6]:
a = 10 * setting_array[0]
b = a + 10 * setting_array[1]
c = b + 10 * setting_array[2]


print(0,a)
print(a,b)
print(b,c)
print(d,100)

0 80
80 80
80 80
100 100


In [13]:
setting = str(setting_array).replace(",","").replace(" ","").replace("[","").replace("]","")

'8002'

In [10]:
with open('temp.txt', 'a+') as f:
    print('printing to a file.', file=f)

In [3]:
import winsound

In [11]:
duration = 1000  # milliseconds
freq = 940  # Hz
winsound.Beep(freq, 500)
winsound.Beep(38, 500)
winsound.Beep(freq, 500)
winsound.Beep(38, 500)
winsound.Beep(freq, 500)
winsound.Beep(38, 500)
winsound.Beep(freq, 500)
winsound.Beep(38, 500)
winsound.Beep(freq, 500)
winsound.Beep(38, 500)
winsound.Beep(freq, 500)
winsound.Beep(38, 500)
